In [ ]:
from dash import Dash, html, dcc,no_update
import plotly.graph_objects as go
from dash.dependencies import Input, Output, State
from tensorflow import keras,data as tfdata
from numpy import load,polyfit,linspace,poly1d
import json
from pandas import read_csv,DataFrame,merge
from ast import literal_eval
import dash_bootstrap_components as dbc
from os import path
my_dir = path.dirname(__file__)

o_player_sequences, d_player_sequences = load(f'{my_dir}/../Notebooks/trajectories_demo.npy',allow_pickle=True)
sequences_ordered = load(f'{my_dir}/../Notebooks/sequences_ordered_demo.npy',allow_pickle=True)
# getting the ordered sequences from the clustering
o_player_sequences_id = load(f'{my_dir}/../Notebooks/o_player_sequences_id_demo.npy',allow_pickle=True)

with open(f'{my_dir}/../Notebooks/player_pairs_demo.json', 'r') as f:
    player_pairs_str = json.load(f)
    player_pairs = {literal_eval(key_str): value for key_str, value in player_pairs_str.items()}

with open(f'{my_dir}/../Notebooks/player_pair_ids_demo.json', 'r') as f:
    player_pair_ids_str = json.load(f)
    player_pair_ids = {literal_eval(key_str): value for key_str, value in player_pair_ids_str.items()}

player_pair_ids = {k:v for k,v in player_pair_ids.items() if v}

trajectory_dict_keys = DataFrame(list(player_pair_ids.keys()),columns=["gameId","playId"])
bdb_games = read_csv(f'{my_dir}/../NFLData/games.csv')
bdb_plays = read_csv(f'{my_dir}/../NFLData/plays.csv')
bdb_players = read_csv(f'{my_dir}/../NFLData/players.csv')
valid_games = trajectory_dict_keys[["gameId"]].drop_duplicates()
valid_plays = trajectory_dict_keys[["gameId","playId"]].drop_duplicates()

valid_games = merge(bdb_games,valid_games,on="gameId",how="inner")
valid_plays = merge(bdb_plays,valid_plays,on=["gameId","playId"],how="inner")


with open(f'{my_dir}/../Notebooks/all_trajectory_dict_demo.json', 'r') as f:
    all_trajectory_dict_1_json = json.load(f)
    all_trajectory_dict = {literal_eval(key_str): value for key_str, value in all_trajectory_dict_1_json.items()}


o_padding_value = [-1.0 for _ in range(len([o_player_sequences[0][0]]))]
padded_o_seq = keras.preprocessing.sequence.pad_sequences(o_player_sequences,padding='post', value=o_padding_value, dtype='float32',maxlen = 90)
masking_o_layer = keras.layers.Masking(mask_value=-1)
masked_o_seq = masking_o_layer(padded_o_seq)

padding_value = [-1.0 for _ in range(len([d_player_sequences[0][0]]))]
padded_d_seq = keras.preprocessing.sequence.pad_sequences(d_player_sequences,padding='post', value=padding_value, dtype='float32',maxlen = 90)
masking_layer = keras.layers.Masking(mask_value=-1)
masked_d_seq = masking_layer(padded_d_seq)
prev_clicks = 0
o_x, o_y,d_x,d_y = [],[],[],[]
los_x_arr,los_y_arr = [],[]
o_marker_colors = ['rgb(200, 200, 255)']
d_marker_colors = ['rgb(200, 255, 200)']
cluster_options = [{'label': f'Cluster Center {i+1}', 'value': i} for i in range(8)]
cluster_center_options = [{'label': f'Sequence {i+1}', 'value': i} for i in range(8)]
game_options = []
for row in valid_games.itertuples():
    game_options.append({'label':f'{row.homeTeamAbbr} vs. {row.visitorTeamAbbr} playing home to {row.homeTeamAbbr} on {row.gameDate}','value':row.gameId})